Import library dan paket yang diperlukan untuk topic modelling (terdiri atas pandas, numpy, re, string, spacy, gensim)

In [1]:
import pandas as pd
import numpy as np

import re
import string

import spacy
import gensim
from gensim import corpora

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=d04bf1465c9384708ee82f08837579641129b683b0fc63d535726e0794f45fe1
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=948e3aa4c9a310f6cefab96ecbc50218e6896e090be4debdec23bf23c24fe838
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyldavis sklearn


Import paket yang diperlukan

In [4]:
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


Kemudian, import juga drive agar data bisa diambil dari google drive.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
hotel_review = pd.read_csv('./drive/My Drive/Colab Notebooks/nlp/UAS/tripadvisor_hotel_reviews.csv')

Membaca isi variabel hotel_review dan melihat bagian teratas pada isi file tersebut.

In [7]:
hotel_review.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


Membuat fungsi untuk cleaning data

In [8]:
def clean_text(text): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()

Download paket stopwords (menghilangkan kakat-kata yang tidak punya makna) pada nltk

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Melakukan proses cleaning data pada fitur Text, kemudian menambahkan fitur baru dengan nama Num_words_text yang berisi jumlah kata pada fitur Text

In [10]:
hotel_review['Text'] = hotel_review['Review'].apply(clean_text)
hotel_review['Num_words_text'] = hotel_review['Text'].apply(lambda x:len(str(x).split())) 

Membuat variabel baru dengan nama max_hotel_review_data_sentence_length yang berisi jumlah kata tertinggi pada fitur Num_words_text. Kemudian membuat variabel baru dengan nama mask dimana isi variabel tersebut berisi kata-kata yang jumlahnya kurang dari 100 dan lebih dari sama dengan 20 kata.

In [11]:
max_hotel_review_data_sentence_length  = hotel_review['Num_words_text'].max()

mask = (hotel_review['Num_words_text'] < 100) & (hotel_review['Num_words_text'] >=20)
df_short_hotel_reviews = hotel_review[mask]

Melihat hasil teratas fiitur Text yang telah di cleaning

In [12]:
df_short_hotel_reviews.head()

,Review,Rating,Text,Num_words_text
0,nice hotel expensive parking got good deal sta...,4,nice hotel expensive parking good deal stay ho...,79
3,"unique, great stay, wonderful time hotel monac...",5,unique great stay wonderful time hotel monaco ...,80
6,"cozy stay rainy city, husband spent 7 nights m...",5,cozy stay rainy city husband spent nights mona...,94
7,"excellent staff, housekeeping quality hotel ch...",4,excellent staff housekeeping quality hotel cho...,80
8,"hotel stayed hotel monaco cruise, rooms genero...",5,hotel stayed hotel monaco cruise rooms generou...,54


Melihat jumlah data sebelum dilakukan cleaning

In [13]:
len(hotel_review.index)

20491

Melihat jumlah data setelah dilakukan cleaning

In [14]:
len(df_short_hotel_reviews.index)

14043

Membuat fungsi remove_stopwrds yang berfungsi untuk melakukan pemecahan pada fitur Text dan menghilangkan kata-kata yang tidak penting dalam bahasa inggris. 

In [15]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
df_short_hotel_reviews['Text']=df_short_hotel_reviews['Text'].apply(remove_stopwords)

<ipython-input-15-1fe61169ace4>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_short_hotel_reviews['Text']=df_short_hotel_reviews['Text'].apply(remove_stopwords)


Melihat isi data setelah dihilangkan stopwords-nya dan dilakukan splitting data.

In [16]:
df_short_hotel_reviews.head()

,Review,Rating,Text,Num_words_text
0,nice hotel expensive parking got good deal sta...,4,nice hotel expensive parking good deal stay ho...,79
3,"unique, great stay, wonderful time hotel monac...",5,unique great stay wonderful time hotel monaco ...,80
6,"cozy stay rainy city, husband spent 7 nights m...",5,cozy stay rainy city husband spent nights mona...,94
7,"excellent staff, housekeeping quality hotel ch...",4,excellent staff housekeeping quality hotel cho...,80
8,"hotel stayed hotel monaco cruise, rooms genero...",5,hotel stayed hotel monaco cruise rooms generou...,54


Import paket ec_core_web_sm dan membuat fungsi lemmatization, berfungsi untuk membagi pada fitur Text menjadi kata kerja (NOUN) dan kata sifat (ADJECTIVE)

In [17]:
#nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])
import en_core_web_sm
nlp = en_core_web_sm.load()
def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

In [18]:
text_list=df_short_hotel_reviews['Text'].tolist()
print(text_list[1])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[1])

unique great stay wonderful time hotel monaco location excellent short stroll main downtown shopping area friendly room showed signs animal hair smells monaco suite sleeping area striped curtains pulled closed nice touch felt cosy goldfish named brandi enjoyed partake free wine coffeetea service lobby thought great feature great staff friendly free wireless internet hotel worked suite laptops decor lovely eclectic pattens color palatte animal print bathrobes feel like rock stars nice look like sterile chain hotel hotel personality excellent stay
['unique', 'great', 'wonderful', 'time', 'hotel', 'location', 'excellent', 'short', 'stroll', 'main', 'downtown', 'shopping', 'area', 'friendly', 'room', 'sign', 'animal', 'hair', 'area', 'curtain', 'nice', 'touch', 'cosy', 'brandi', 'partake', 'free', 'wine', 'coffeetea', 'service', 'lobby', 'great', 'feature', 'great', 'staff', 'friendly', 'free', 'wireless', 'internet', 'hotel', 'lovely', 'eclectic', 'patten', 'color', 'palatte', 'animal', '

In [19]:
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

Membuat pemodelan dengan LDA berdasarkan jumlah topik yang diberikan (misalkan disini ditentukan 7 topik)

In [20]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=7, random_state=100,
                chunksize=1000, passes=50,iterations=250)

Output streaming akan dipotong hingga 5000 baris terakhir.
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np

Melihat topik yang telah ditentukan sebelumnya

In [ ]:
lda_model.print_topics()

[(0,
  '0.012*"prompt" + 0.012*"deceptive" + 0.009*"club" + 0.008*"partner" + 0.008*"hospitality" + 0.008*"signage" + 0.007*"pm" + 0.006*"local" + 0.006*"eating" + 0.006*"true"'),
 (1,
  '0.052*"hotel" + 0.031*"room" + 0.014*"desk" + 0.012*"service" + 0.011*"staff" + 0.009*"night" + 0.009*"place" + 0.009*"bad" + 0.008*"manager" + 0.008*"day"'),
 (2,
  '0.087*"hotel" + 0.038*"room" + 0.033*"great" + 0.026*"staff" + 0.024*"location" + 0.024*"good" + 0.015*"breakfast" + 0.014*"helpful" + 0.014*"clean" + 0.014*"night"'),
 (3,
  '0.077*"room" + 0.042*"hotel" + 0.018*"nice" + 0.017*"good" + 0.015*"night" + 0.013*"floor" + 0.012*"bathroom" + 0.012*"small" + 0.011*"breakfast" + 0.010*"area"'),
 (4,
  '0.032*"great" + 0.030*"food" + 0.030*"pool" + 0.027*"beach" + 0.022*"resort" + 0.021*"time" + 0.019*"good" + 0.017*"beautiful" + 0.016*"service" + 0.013*"staff"'),
 (5,
  '0.012*"language" + 0.012*"villa" + 0.011*"country" + 0.011*"sick" + 0.007*"blast" + 0.007*"steam" + 0.006*"complex" + 0.005*"

Visualilasi daftar topik yang telah ditentukan sebelumnya

In [21]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.279702 -0.033496       1        1  47.515247
3      0.250770  0.050029       2        1  25.565090
4      0.167548 -0.206932       3        1  13.297547
1      0.095849  0.228479       4        1   7.451191
5     -0.266910 -0.014989       5        1   2.134149
6     -0.266969  0.058706       6        1   2.099623
0     -0.259990 -0.081798       7        1   1.937154, topic_info=        Term          Freq         Total Category  logprob  loglift
23     hotel  29028.000000  29028.000000  Default  30.0000  30.0000
43      room  21302.000000  21302.000000  Default  29.0000  29.0000
20     great  11633.000000  11633.000000  Default  28.0000  28.0000
144     food   2925.000000   2925.000000  Default  27.0000  27.0000
413     pool   2434.000000   2434.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
1440    wide     43.214463    130.149739   Topic7  -5.4523   2.8414
55     chain     44.508303    185.697610   Topic7  -5.4228   2.5155
3487  museum     44.449811    255.618163   Topic7  -5.4241   2.1946
422    pizza     41.381822     92.520079   Topic7  -5.4957   3.1394
1848    odor     40.287340    149.308451   Topic7  -5.5225   2.6340

[419 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
2690       3  0.994617       activity
2066       4  0.993163         actual
4626       7  0.976478  advertisement
898        1  0.998547        advisor
23391      7  0.971958         aerate
...      ...       ...            ...
3557       4  0.986205     wonderfull
438        1  0.453129           year
438        2  0.121166           year
438        3  0.380927           year
438        4  0.044815           year

[583 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 5, 2, 6, 7, 1])

In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Output streaming akan dipotong hingga 5000 baris terakhir.
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np